In [5]:
from IPython.display import YouTubeVideo

# MSE 8900 / CHE 8450 - Multiscale Modeling

Rachel B. Getman, Sapna Sarupria, Ulf D. Schiller

Clemson University

## Lecture 3.5: Lattice Boltzmann Methods

1. The lattice Boltzmann equation
2. Fluctuating lattice Boltzmann
3. Chapman-Enskog expansion
4. LB-MD coupling
5. Boundary conditions

### References
1. Sauro Succi, The Lattice Boltzmann Equation for Complex States of Flowing Matter, Oxford University Press, 2018.  
2. B. Duenweg and A. J. C. Ladd, Lattice Boltzmann Simualtions of Soft Matter, Adv. Polym. Sci. 221, 89, 2009.

### From Newton to Navier-Stokes

![Route](assets/route.png)

### Kinetic Theory: Boltzmann Equation

* evolution equation for the (one-)particle distribution function
$$
\left( \frac{\partial}{\partial t} + \vec{v}\cdot\frac{\partial}{\partial\vec{r}} + \frac{\vec{F}}{m}\cdot\frac{\partial}{\partial\vec{v}} \right) f(\vec{r},\vec{v},t) = \mathcal{C}[f]
$$

* Boltzmann collision operator
$$
\mathcal{C}[f] = \int d\vec{v}_1 \int d\Omega \, \sigma(v_\text{rel},\Omega) \, v_\text{rel} \left[ f(\vec{r},\vec{V}',t) f(\vec{r},\vec{v}_1',t) - f(\vec{r},\vec{v},t) f(\vec{r},\vec{v}_1,t) \right] 
$$

* Detailed balance
$$
f(\vec{r},\vec{v}_1',t) f(\vec{r},\vec{v}_2',t) = f(\vec{r},\vec{v}_1,t) f(\vec{r},\vec{v}_2,t)
$$

* Equilibrium distribution (Maxwell-Boltzmann) $f=f^\text{eq} + f^\text{neq}$
\begin{equation*}
\ln f^\text{eq} = \gamma_0 + \gamma\,\vec{v} + \gamma_4 \vec{v}^2
\end{equation*}



### Hydrodynamic Fields

* "average" of polynomials $\psi(\vec{v})$ in components of $\vec{v}$
$$\begin{aligned}
m_\psi(\vec{r},t) = \int \psi(\vec{v}) \, f(\vec{r},\vec{v},t) d\vec{v}
\end{aligned}$$

* density, momentum density, stress tensor
$$\begin{aligned}
\rho(\vec{r}, t) &= m \int f d\vec{v} \\
\vec{j}(\vec{r}, t) &= m \int \vec{v} f d\vec{v} \\
\Pi(\vec{r} ,t) &= m \int \vec{v} \otimes \vec{v} f d\vec{v}
\end{aligned}$$


### Separation of scales

* Observation: not all $m_\psi$ show up in the macroscopic equations of motion

* $\rho$, $\vec{j}$ (and $e$) are collisional invariants
\begin{equation*}
\int d\vec{r} d\vec{v} \frac{\delta m_{\rho,\vec{j},e}(f)}{\delta f} \mathcal{C}[f] = 0
\end{equation*}

* local equilibrium (Maxwell-Boltzmann) $f^\text{eq}(\rho,\vec{j},e)$

* Hydrodynamics describes variation of $\rho$ and $\vec{j}$ (and $e$) through \emph{transport} (over a macroscopic distance $\sim L$)

* all other variables relax rapidly through \emph{collisions} ($\sim \lambda$ mean free path)

* scale separation: $\epsilon \sim Kn = \frac{\lambda}{L} \ll 1$ \qquad Knudsen number $Kn=\frac{\lambda}{L}$

#### How can we exploit the scale separation?

* we are only interested in the dynamics of the slow variables up to a certain order
* the dynamics of the fast variables beyond that order is unimportant
* any set of fast variables that leaves the slow dynamics unchanged will do
* the number of degrees of freedom can be greatly reduced!
* Caveat: imperfect scale separation $\rightarrow$ fast variables can couple to slow dynamics

![Box](assets/box.png)

### Discretization a la Grad

* Truncated Hermite expansion
$$f^N(\vec{r},\vec{v},t)=\omega(\vec{v}) \sum_{n=0}^N \frac{1}{n!}\mathsf{a}^{(n)}(\vec{r},t) \mathcal{H}^{(n)}(\vec{v})$$

($a^{(0)}=\rho$, $a^{(1)}=\vec{j}$, $a^{(2)} = \Pi - \rho\mathsf{I}$, $\dots$)

* Gauss-Hermite quadrature
$$\begin{aligned}
\mathsf{a}^{(n)} &= \int \mathcal{H}^{(n)}(\vec{v}) f^N(\vec{r},\vec{v},t) \, d\vec{v} 
= \sum_{i} w_i \, \frac{\mathcal{H}^{(n)}(\vec{c}_i) f^N(\vec{r},\vec{c}_i,t)}{\omega(\vec{c}_i)}\\
&= \sum \mathcal{H}^{(n)}(\vec{c}_i) f_i(\vec{r},t)
\end{aligned}$$

$$\int \omega(\vec{v}) \, p(\vec{r},\vec{v},t) \, d\vec{v} = \sum_{i} w_i \, p(\vec{r},\vec{c}_i,t)$$

* Discrete velocity model (DVM)

$$\left( \frac{\partial}{\partial t} + \vec{c}_i \cdot \frac{\partial}{\partial\vec{r}} \right) f_i = - \sum_j \Omega_{ij} (f_j - f_j^\text{eq})$$

### Quadratures

| Quadrature | LB model | $q$ | $b_q$ | $w_q$ | $\vec{c}_q$ |
|---|---|---|---|---|---|
| $E_{1,5}^{3}$ | D1Q3 | 0 | 1 | $\frac{2}{3}$  | 0 |
|               |      | 1 | 2 | $\frac{1}{6}$  | $\pm\sqrt{3}$ |
| $E_{2,5}^{9}$ | D2Q9 | 0 | 1 | $\frac{4}{9}$  | $(0,0)$ |
|               |      | 1 | 4 | $\frac{1}{9}$  | $(\pm\sqrt{3},0)$, $(0,\pm\sqrt{3})$ |
|               |      | 2 | 4 | $\frac{1}{36}$ | $(\pm\sqrt{3},\pm\sqrt{3})$ |
| $E_{3,5}^{15}$ | D3Q15 | 0 | 1 | $\frac{2}{9}$ | $(0,0,0)$ |
|                | | 1  |  6 | $\frac{1}{9}$  | $(\pm\sqrt{3},0,0)$, $(0,\pm\sqrt{3},0)$, $(0,0,\sqrt{3})$ |
|                | | 3  |  8 | $\frac{1}{72}$ | $(\pm\sqrt{3},\pm\sqrt{3},\pm\sqrt{3})$ |
| $E_{3,5}^{19}$ | D3Q19 | 0 | 1 | $\frac{1}{3}$ | $(0,0,0)$ |
|                | | 1 |  6 | $\frac{1}{18}$ | $(\pm\sqrt{3},0,0)$, $(0,\pm\sqrt{3},0)$, $(0,0,\sqrt{3})$ |
|                | | 2 | 12 | $\frac{1}{36}$ | $(\pm\sqrt{3},\pm\sqrt{3},0)$, $(\pm\sqrt{3},0,\pm\sqrt{3})$, $(0,\pm\sqrt{3},\pm\sqrt{3})$ |
| $E_{3,5}^{27}$ | D3Q27 | 0 | 1 | $\frac{8}{27}$ | $(0,0,0)$ |
|                | | 1 |  6 | $\frac{2}{27}$ | $(\pm\sqrt{3},0,0)$, $(0,\pm\sqrt{3},0)$, $(0,0,\sqrt{3})$ |
|                | | 2 | 12 | $\frac{1}{54}$ | $(\pm\sqrt{3},\pm\sqrt{3},0)$, $(\pm\sqrt{3},0,\pm\sqrt{3})$, $(0,\pm\sqrt{3},\pm\sqrt{3})$ |
|                | | 3 |  8 | $\frac{1}{216}$ | $(\pm\sqrt{3},\pm\sqrt{3},\pm\sqrt{3})$

Notation $E^n_{D,d}$: $D$ dimensions, $d$ degree, $n$ abscissae
$q$: neighbor shell, $b_q$: number of neighbors, $w_q$ weights, $\vec{c}_q$ velocities

\begin{equation*}
\mathsf{T}^{(n)} = \sum_i w_i \vec{c}_i \dots \vec{c}_i = 
\begin{cases}
0 & n \text{ odd} \\
{\delta}^{(n)} & n \text{ even}
\end{cases}, \quad \forall n \leq d.
\end{equation*}

### Models with polynomial equilibrium

* Ansatz: expansion in the velocities $\vec{u}$ (Euler stress is quadratic in $\vec{u}$)
\begin{equation*}
f_i^\text{eq}(\rho,\vec{u}) = w_i \rho \left[ 1 + A \vec{u}\cdot\vec{c}_i + B (\vec{u}\cdot\vec{c}_i)^2 + C\vec{u}^2 \right]
\end{equation*}

* cubic symmetry of lattice tensors $\mathsf{T}^{(n)}$
  \begin{alignat*}{2}
    &\sum_i w_i = 1 &
    &\sum_i w_i c_{i \alpha} = 0 \\
    &\sum_i w_i c_{i \alpha} c_{i \beta}
       =  \sigma_2 \, \delta_{\alpha \beta} &
    &\sum_i w_i c_{i \alpha} c_{i \beta} c_{i \gamma} = 0 \\
    &\sum_i w_i c_{i \alpha} c_{i \beta} c_{i \gamma} c_{i \delta}
      =  \kappa_4 \, \delta_{\alpha \beta \gamma \delta} &&
      + \sigma_4  \left(
          \delta_{\alpha \beta}  \delta_{\gamma \delta} 
        + \delta_{\alpha \gamma} \delta_{\beta \delta}
        + \delta_{\alpha \delta} \delta_{\beta \gamma}
        \right)
\end{alignat*}

* at least three shells required to satisfy the conditions
\begin{alignat*}{4}
\sum_i w_i &= 1 & \qquad
\kappa_4 &= 0 & \qquad
\sigma_4 &= \sigma_2^2 & \qquad
c_s^2 &= \sigma_2 
\end{alignat*}

### The D3Q19 model

![D3Q19 Model](assets/d3q19.png)

* Equilibrium distribution:
\begin{equation*}
f_i^\text{eq}(\rho,\vec{u}) = w_i \rho \left[ 1 + \frac{\vec{u}\cdot\vec{c}_i}{c_s^2} + \frac{\vec{u}\vec{u} \colon (\vec{c}_i\vec{c}_i - c_s^2 \mathsf{I})}{2c_s^4} \right]
\end{equation*}

* Moments:
\begin{align*}
\sum_i f_i^\text{eq} &= \rho \\
\sum_i f_i^\text{eq}\vec{c}_i &= \rho\vec{u} \\
\sum_i f_i^\text{eq}\vec{c}_i\vec{c}_i &= \rho c_s^2 \mathsf{I} + \rho\vec{u}\vec{u}
\end{align*}

* Weight coefficients:
\begin{equation*}
w_i =1/3 \quad \text{for } |\vec{c}_i|=0, \qquad
w_i = 1/18 \quad \text{for } |\vec{c}_i|=1, \qquad
w_i = 1/36 \quad \text{for } |\vec{c}_i|=\sqrt{2}
\end{equation*}
* Speed of sound: $c_s = \frac{1}{\sqrt3}\left(\frac{a}{h}\right)$

### The lattice Boltzmann equation

* recall the Boltzmann equation with linearised collisions

\begin{equation*}
\left( \frac{\partial}{\partial t} + \vec{v} \cdot \frac{\partial}{\partial\vec{r}} \right) f(\vec{r},\vec{v},t) = - \Omega \left[ f(\vec{r},\vec{v},t) - f^{\text{eq}}(\vec{v}) \right]
\end{equation*}

* systematic discretization $\rightarrow$ lattice Boltzmann equation

\begin{equation*}
\bar f_i(\vec{r}+h\vec{c}_i,t+h) = \bar f_i^*(\vec{r},t) = \bar f_i(\vec{r},t) + \sum_j \Lambda_{ij} \left[ \bar f_j(\vec{r},t) - f_j^{\text{eq}}(\rho,\vec{u}) \right]
\end{equation*}

* But this is only first-order in time!?

### Splitting of the discrete Boltzmann equation

* Discrete velocity model (now with force term)

\begin{equation*}
%\frac{df_i}{dt} + {\lambda} f_i = {\lambda} f_i^\text{eq}.
\frac{\partial}{\partial t} f_i = - \vec{c}_i \cdot \frac{\partial}{\partial\vec{r}} f_i - \sum_j \Omega_{ij} (f_j - f_j^\text{eq}) + G_i = \left( \mathcal{S} + \mathcal{C} + \mathcal{F} \right) f_i
\end{equation*}

* Formal solution

$$f_i(t+h) = \exp\left[ {h(\mathcal{S} + \mathcal{C} + \mathcal{F})} \right] \, f_i(t)$$

* Baker-Campbell-Hausdorff (for sufficiently differentiable operators)

\begin{equation*}
\exp \left[ h(\mathcal{S} + \mathcal{C} + \mathcal{F}) \right] = e^{ \frac{h}{2} \left( \mathcal{C} + \mathcal{F} \right) } e^{  h \mathcal{S} } e^{ \frac{h}{2} \left( \mathcal{C} + \mathcal{F} \right) } + \mathcal{O}(h^3) \approx \mathsf{C}^\frac12 \mathsf{S} \mathsf{C}^\frac12
\end{equation*}

* Second-order in time lattice Boltzmannn update

\begin{equation*}
\mathbf{f}(t+h) = \mathsf{C}^\frac12 \mathsf{S} \mathsf{C}^\frac12 \, \mathbf{f}(t)
\end{equation*}

P. Dellar, Comp. Math. App. 65, 129-141 (2013)  
UDS, Comp. Phys. Comm. 185, 2586-2597 (2014)

### Splitting of the discrete Boltzmann equation

* Streaming (Integration along characteristic)

\begin{equation*}
f_i(\vec{r}+\vec{c}_i,t+h) = f_i(\vec{r},t)
\end{equation*}

* Collisions (Crank-Nicolson rule)

\begin{equation*}
  \mathbf{f}^\text{nq}(t+h) = \left( \mathsf{I} + \frac{h}{2} \mathsf{\Omega} \right)^{-1} 
  \left( \mathsf{I} - \frac{h}{2} \mathsf{\Omega} \right) \mathbf{f}^\text{nq}(t) + \mathcal{O}((h/\tau)^3)
\end{equation*}

* The error $\mathcal{O}((h/\tau)^3)$ can cause non-linear instabilities!

* Why not exact solution?

\begin{equation*}
  \mathbf{f}^\text{nq}(t+h) = \exp [ - h \mathsf{\Omega} ] \, \mathbf{f}^\text{nq}(t)
\end{equation*}

* in practice less accurate!

P. Dellar, Comp. Math. App. 65, 129-141 (2013)  
Brownlee et al., Phys. Rev. E 75, 036711 (2007)

### The classical LB algorithm

1. streaming step: move $f_i^*(\vec{r},t)$ along $\vec{c}_i$ to the next lattice site, increment $t$ by $h$

$$
f_i(\vec{r}+h\vec{c}_i,t+h) = f_i^*(\vec{r},t)
$$

2. collision step: apply $\Lambda_{ij}$ and compute the post-collisional $f_i^*(\vec{r},t)$ on every lattice site

$$
f_i^*(\vec{r},t) = f(\vec{r},t) + \sum_j \Lambda_{ij} \left[ f_j(\vec{r},t) - f_j^\text{eq}(\rho,\vec{u}) \right]
$$

### Macroscopic moments in lattice Boltzmann

* macroscopic fields are velocity moments of the populations

\begin{align*}
\rho &= \sum_i f_i & \rho\vec{u} &= \sum_i f_i\vec{c}_i+\frac{h}{2}\mathbf{f} & \mathsf{\Pi} &= \sum_i \frac{f_i+f_i^*}{2} \vec{c}_i\otimes\vec{c}_i
\end{align*}

* construct orthogonal basis $e_{ki}$ for moments (recall $\psi(\vec{v})$ and $m_\psi$)
\begin{equation*}
m_k = \sum_i e_{ki} f_i
\end{equation*}

$0 \leq k \leq 9$: hydrodynamic modes (slow), $k\geq10$: kinetic modes (fast)

* collision matrix is diagonal in mode space
\begin{equation*}
\Lambda (\mathbf{f} - \mathbf{f}^\text{eq}) = \mathsf{M}^{-1} {\left( \mathsf{M}\Lambda\mathsf{M}^{-1} \right)}{ \mathsf{M} (\mathbf{f} - \mathbf{f}^\text{eq})} = \mathsf{M}^{-1} {\hat\Lambda}{ (\mathbf{m}-\mathbf{m}^\text{eq}) }
\end{equation*}

* MRT model
\begin{align*}
(m_k - m_k^\text{eq})^* = \gamma_k (m_k - m_k^\text{eq})
\end{align*}

### Choice of the moment basis

\begin{align*}
& m_0 = \rho = \sum_i f_i       && \text{mass}\\
& m_1 = j_x = \sum_i f_i c_{ix} && \text{momentum $x$}\\
& m_2 = j_y = \sum_i f_i c_{iy} && \text{momentum $y$}\\
& m_3 = j_z = \sum_i f_i c_{iz} && \text{momentum $z$}\\
& m_4 = \text{tr}(\mathsf{\Pi})             && \text{bulk stress}\\
& m_5, \ldots, m_9  \simeq \overline{\mathsf{\Pi}} && \text{shear stresses}\\
& m_{10}, \ldots, m_{18}        && \text{"kinetic modes", "ghost modes"}\\
\end{align*}

### Multiple relaxation time model (MRT)

\begin{align*}
&\gamma_0 = \gamma_1 = \gamma_2 = \gamma_3 = 0 \quad \text{mass and momentum conservation} \\
&\gamma_4 = \gamma_b \quad \text{bulk stress} \\
&\gamma_5 = \ldots = \gamma_9 = \gamma_s \quad \text{shear stress} \\
&\gamma_{10} = \ldots = \gamma_{18} = 0 \quad \text{simplest choice, careful with boundaries!}
\end{align*}

* Remark: we could also relax the populations directly:
\begin{equation*}
f_i^{\text{nq}*} = \sum_j \Lambda_{ij} f_j^\text{nq}
\end{equation*}
* simplest choice $\Lambda_{ij}=\lambda^{-1}\delta_{ij}$ $\rightarrow$ lattice BGK 
* not a particularly good choice (less stable, less accurate)

### Lattice moments vs. hydrodynamic moments

* Concatenation of lattice Boltzmann updates

\begin{equation*}
\mathbf{f}(t+N h) = \mathsf{C}^\frac12 \left( \mathsf{S} \mathsf{C} \right)^N \mathsf{C}^{-\frac{1}{2}} \, \mathbf{f}(t)
\end{equation*}

* density
$$
\rho = \sum_i ( \mathsf{C}^\frac12 \mathbf{f} )_i = \sum_i f_i
$$

* momentum density
$$
\vec{j} = \sum_i ( \mathsf{C}^\frac12 \mathbf{f} )_i \vec{c}_i 
= \sum_i f_i \vec{c}_i + \frac{h}{2} \mathbf{f}
$$

* viscous stress

\begin{equation*}
\begin{split}
\sigma &= - \sum_i ( \mathsf{C}^\frac12 \mathbf{f}^\text{nq} )_i \vec{c}_i\vec{c}_i
%= \mathsf{C}^\frac12 \Pi^\text{nq}  \\
%&\approx \frac{1}{2} \left( \1 + \mathsf{C} \right) \Pi^\text{nq} 
= - \left( \mathsf{I}+ \frac{h}{2} \Omega \right)^{-1} \sum_i f^\text{nq}_i \vec{c}_i\vec{c}_i %\Pi^\text{nq}
\end{split}
\end{equation*}

* {Note: if $\mathsf{C}^\frac12$ includes fluctuations, it may not be invertible!}

### Viscous stress relaxation

$$ {\Pi} = \overline{{\Pi}} + \frac{1}{3} \text{tr}({\Pi}) \mathsf{I}$$

* recall: collision step applies linear relaxation to the moments

\begin{eqnarray*}
\overline{{\Pi}}^{*\text{neq}} &=& \gamma_s \overline{{\Pi}}^{\text{neq}} \\
\text{tr}({\Pi}^{*\text{neq}}) &=& \gamma_b \text{tr}({\Pi}^{\text{neq}})
\end{eqnarray*}

* Chapman-Enskog expansion

\begin{equation*}
-\frac{1}{2} \left ( \mathsf{\Pi}^{*\text{neq}} + \mathsf{\Pi}^\text{neq} \right) = \sigma = \eta \left( \nabla\vec{u} + (\nabla\vec{u})^t \right) + \left( \zeta - \frac{2}{3} \eta \right) (\nabla\cdot\vec{u}) \mathsf{I}
\end{equation*}

* shear and bulk viscosities are determined by the relaxation parameters

\begin{align*}
\eta &= \frac{\rho c_s^2 h}{2} \frac{1+\gamma_s}{1-\gamma_s}& \eta_b &= \frac{\rho c_s^2 h}{3} \frac{1+\gamma_b}{1-\gamma_b}
\end{align*}

### Viscosity of the lattice Boltzmann fluid

* incompressible Navier-Stokes equation is recovered

* $-1 \leq \gamma_s \leq 1$ $\Leftrightarrow$ positive viscosities

* any viscosity value is accessible

![LB Viscosity](assets/visc_vs_gamma.png)

### The force term in LB

* force term $G_i$ in the discrete velocity model has the same
  first moments as the acceleration term $-\frac{1}{\rho} \vec{F} \cdot
  \nabla_\vec{c} f$

\begin{align*}\label{eq:force-moments}
\sum_i G_i &= 0, & \sum_i \vec{c}_i G_i &= \vec{F}, & \sum_i \vec{c}_i \vec{c}_i G_i &= \vec{F} \vec{u} + \vec{u} \vec{F},
\end{align*}

* second-order accuracy requires to transform the force term

\begin{equation*}
\bar{\vec{G}} = \left( \mathsf{I} + \frac{h}{2} \Omega \right)^{-1} h \Omega \, \vec{G} = \left( \mathsf{I} - \frac12 \Lambda \right) \vec{G}
\end{equation*}

* leads to a self-consistency problem when forces are velocity dependent

* collide-stream-collide scheme

### Units in LB

* grid spacing $a$, time step $h$, particle mass $m_p$

* these merely control the _accuracy_ and _stability_ of LB!

* physically relevant: mass density $\rho$, viscosity $\eta$, temperature $k_BT$

* recall:

\begin{align*}
c_s^2 &= \frac{1}{3} \frac{a^2}{h^2} = \hat{c}_s^2 \frac{{a^2}}{{h^2}}\\
\eta &= \frac{\rho c_s^2 h}{2} \frac{1+\gamma_s}{1-\gamma_s} = \hat{\rho} \hat{c}_s^2 \hat{\eta} \frac{m_p}{{a}{h}} \\
k_BT &= m_p c_s^2 = m_p \hat{c}_s^2 \frac{{a^2}}{{h^2}}
\end{align*}

* always make sure you are simulating the right _physics_!

* for comparison with experiments: match dimensionless numbers!
($Ma$, $Re$, $Pe$, $Sc$, $Kn$, $Pr$, $Wi$, $De$, ...)

### Dimensionless numbers and LB parameters

* connection between dimensionless numbers and simulation parameters

\begin{equation*}
\frac{1+\gamma}{1-\gamma} = \frac{2\nu}{c_s^2 h} = 2 \sqrt{3} \hat{L} \frac{Ma}{Re} > 0
\end{equation*}

* Knudsen number

\begin{equation*}
Kn\propto\frac{Ma}{Re} = \frac{\eta}{\rho c_s L} = \frac{c_s h}{2 L} \frac{1 + \gamma}{1 - \gamma} = \frac{1}{\sqrt{3} \hat L} \left( \frac{1}{\lambda} - \frac{1}{2} \right)
\end{equation*}

* diffusive scaling: $a \sim \epsilon L$ \quad $h \sim \epsilon^2 T$

\begin{align*}
\frac{{a^2}}{{h}} &= \text{const.} &
\frac{{a}}{{h}} \sim \frac{1}{\epsilon}\frac{L}{T} \underset{\epsilon\rightarrow 0}{\rightarrow} \infty
\end{align*}

* $Ma \rightarrow 0$ at fixed Reynolds number

* Mach number annealing: $h \sim \epsilon T$

* run simulation with higher Mach number until transients have decayed

### Thermal fluctuations

* so far the LB model is athermal and entirely deterministic

* for Brownian motion, we need fluctuations!

![Velocity decay](assets/vel.png)
![Brownian motion](assets/brownian-motion.png)

#### Do we need fluctuations?

* Ideal gas, temp. $T$, particle mass $m_p$, sound speed $c_s$:

$$  k_B T = m_p c_s^2$$

* $c_s \sim a / h$ ($a$ lattice spacing, $h$ time step)
* $c_s$ as small as possible

* Example (water)

|||
|---|---|
| sound speed realistic:  | $1.5 \times 10^3 m / s$ |
| sound speed artificial: | $c_s = 10^2 m / s$      |
| temperature             | $T \approx 300 K$, $k_B T = 4 \times 10^{-21}$ |
| particle mass:          | $m_p = 4 \times 10^{-25} kg$ |

|   | macroscopic scale | molecular scale |
|---|---|---|
| lattice spacing    | $a = 1 mm$             | $a = 1 nm$             |
| time step          | $h = 10^{-5} s$        | $h = 10^{-11} s$       |
| mass of a site     | $m_a = 10^{-6} kg$     | $m_a = 10^{-24} kg$    |
| "Boltzmann number" | $Bo = (m_p/m_a)^{1/2}$ | $Bo = (m_p/m_a)^{1/2}$ |
|                    | $ = 6 \times 10^{-10}$ | $ = 0.6$               |

### Low Mach number physics

* LB requires $u \ll c_i$ hence $u \ll c_s$
* low Mach number $Ma=u/c_s \ll 1$
* compressibility does not matter
* equation of state does not matter $\rightarrow$ choose ideal gas!

$m_p$ particle mass

\begin{equation*}
  p = \frac{\rho}{m_p} k_B T
\end{equation*}

\begin{equation*}
  c_s^2 = \frac{\partial p}{\partial \rho} = \frac{1}{m_p} k_B T
\end{equation*}

\begin{equation*}
  p = \rho c_s^2
\end{equation*}

\begin{equation*}
  k_B T = m_p c_s^2
\end{equation*}


### Generalized lattice gas model (GLG)

* consider integer population numbers ($m_p$ mass of an LB particle)

\begin{equation*}
\nu_i = \frac{f_i}{\mu} \qquad \mu=\frac{m_p}{a^3} \qquad \mu \nu_i = {w_i}\rho
\end{equation*}

* each lattice site in contact with a heat bath

* Poisson + constraints

\begin{equation*}
     P \left( \left\{ \nu_i \right\} \right)
     \propto
     \prod_i
     \frac{ {\bar \nu}_i^{\nu_i} }{\nu_i !} e^{-{\bar \nu}_i}
     \delta \left( \mu \sum_i \nu_i - \rho \right)
     \delta \left( \mu \sum_i \nu_i \vec c_i - \vec j \right)
\end{equation*}

[B. Duenweg, UDS, A. J. C. Ladd, Phys. Rev. E 76, 036704 (2007)]

### Entropy

* associated entropy
\begin{equation*}
    P % \left( \left\{ \nu_i \right\} \right) 
    \propto
    \exp \left[S \left( \left\{ \nu_i \right\} \right)\right]
    \delta \left( \mu \sum_i \nu_i - \rho \right)
    \delta \left( \mu \sum_i \nu_i \vec c_i - \vec j \right)
\end{equation*}

* Stirling: $\nu_i! = \exp \left( \nu_i \ln \nu_i - \nu_i \right)$
\begin{align*}
    S \left( \left\{ \nu_i \right\} \right) 
    &= - \sum_i \left( \nu_i \ln \nu_i -\nu_i - \nu_i \ln {\bar \nu}_i
    + {\bar \nu}_i \right) \\
    &= \frac{1}{\mu} \sum_i \rho w_i \left( \frac{f_i}{\rho w_i}-\frac{f_i}{\rho w_i} \ln\frac{f_i}{\rho w_i} - 1 \right)
\end{align*}

* $\mu$ controls the mean square fluctuations (degree of coarse-graining)

### Maximum entropy principle

* maximize entropy $S$ subject to constraints for mass and momentum conservation

\begin{equation*}
\frac{\partial S}{\partial \nu_i} + \chi
+ \lambda \cdot \vec c_i  =  0 \qquad
\mu \sum_i \nu_i - \rho  =  0 \qquad
\mu \sum_i \nu_i \vec c_i - \vec{j}  =  0
\end{equation*}

* formal solution

\begin{equation*}
f_i^{\text{eq}} = w_i \, \rho \exp \left( \chi + \lambda \cdot \vec{c}_i \right)
\end{equation*}

* expansion up to $\mathcal{O}(u^2)$

\begin{equation*}
f_i^\text{eq}(\rho,\vec{u}) = w_i \rho \left[ 1 + \frac{\vec{u}\cdot\vec{c}_i}{c_s^2} + \frac{\vec{u}\vec{u} \colon (\vec{c}_i\vec{c}_i - c_s^2 \mathsf{I})}{2c_s^4} \right]
\end{equation*}



### Fluctuations around equilibrium

* Gauss distribution for non-equilibrium part
\begin{equation*}
    P % \left( \left\{ f_i^{neq} \right\} \right)
    \propto
    \exp \left( -  \sum_i \frac{ \left( f_i^{\text{nq}} \right)^2 }
                               { 2 \mu \rho w_i } \right)
    \delta \left( \sum_i f_i^{\text{nq}} \right)
    \delta \left( \sum_i \vec c_i \, f_i^{\text{nq}} \right)
\end{equation*}

* transform to the modes ($b_k = \sum_i w_i e_{ki}^2$, Basis $e_{ki}$)
\begin{equation*}
    P\left( \left\{ {m}_k^{\text{nq}} \right\} \right)
    \propto \exp \left( - \sum_{k \ge 4}
    \frac{\left( {m}_k^{\text{nq}} \right)^2}{2 \mu \rho b_k} \right)
\end{equation*}

* more convenient: ortho-_normal_ modes
\begin{equation*}
    \hat{m}_k  =  \sum_i \hat{e}_{ki} 
    \frac{f_i}{\sqrt{w_i \mu \rho}}
\end{equation*}

### Implementation of the fluctuations

* introduce stochastic term into the collision step
\begin{equation*}
    {m}_k^{* \text{nq}} = \gamma_k {m}_k^{\text{nq}} + \varphi_k r_k
\end{equation*}
$r_k$ random number from normal distribution

* ensure detailed balance (like in Monte-Carlo)
\begin{equation*}
\frac{p(m \rightarrow m^*)}{p(m^* \rightarrow m)} = \frac{\exp(-m^{*2}/2)}{\exp(-m^2/2)} \qquad \Rightarrow \qquad {\varphi_k = \sqrt{\mu\rho b_k \left( 1 - \gamma_k^2 \right)}}
\end{equation*}

* $\varphi_k \neq 0$ for \emph{all} non-conserved modes
* all modes have to be thermalized

[A. J. C. Ladd, J. Fluid Mech. 271, 285-309 (1994)]  
[Adhikari et al., Europhys. Lett. 71, 473-479 (2005)]  
[B. Duenweg, UDS, A. J. C. Ladd, Phys. Rev. E 76, 036704 (2007)]

### Eliminating fast variables: Chapman-Enskog

* introduce length and time scales
\begin{alignat*}{3}
&\text{coarse-grained length:} &\quad& \vec{r}_1 = \epsilon \vec{r} &\quad\rightarrow\quad& \frac{\partial}{\partial\vec{r}}=\epsilon\frac{\partial}{\partial\vec{r}_1} \\
&\text{convective time scale:} &\quad& t_1 = \epsilon t \\
&\text{diffusive time scale:}  &\quad& t_2 = \epsilon^2 t &\quad\rightarrow\quad& \frac{\partial}{\partial t} = \epsilon \frac{\partial}{\partial t_1} + \epsilon^2 \frac{\partial}{\partial t_2}
\end{alignat*}

* use $\epsilon$ as a perturbation parameter and expand $f$
\begin{eqnarray*}
f &=& f^{(0)} + \epsilon f^{(1)} + \epsilon^2 f^{(2)} + \dots \\
\mathcal{C}[f] &=& \mathcal{C}[f^{(0)}] + \epsilon \int d\vec{r} d\vec{v} \frac{\delta C[f]}{\delta f} f^{(1)}(\vec{r},\vec{v}) + \dots
\end{eqnarray*}

* solve for each order in $\epsilon$

### Chapman-Enskog expansion

* $\epsilon^0$: yields the collisional invariants, and the equilibrium distribution $f^{(0)}=f^\text{eq}$

* $\epsilon^1$: yields the Euler equations, and the first order correction $f^{(1)}$

\begin{equation}
\tag{*}
\left( \frac{\partial}{\partial t_1} + \vec{c}\frac{\partial}{\partial\vec{r}_1} \right) f^\text{(0)} = \int d\vec{r}' d\vec{c}' \frac{\delta C[f^{(0)}]}{\delta f^{(0)}} f^{(1)}(\vec{r}',\vec{c}')
\end{equation}

* $\epsilon^2$: adds viscous terms to the Euler equation

* Navier-Stokes!

* the "only" difficulty is: no explicit solution of (*) is known...\\ (except for Maxwell molecules)

### Chapman-Enskog expansion for LB

* original LBE
\begin{equation*}
f_i(\vec{r}+h\vec{c}_i,t+h) - f_i(\vec{r},t) = \delta_i
\end{equation*}

* recall: coarse-grained length $\vec{r}_1$, convective time scale $t_1$, diffusive time scale~$t_2$

\begin{equation*}
f_i(\vec{r}_1+\epsilon h\vec{c}_i,t_1+\epsilon h,t_2+\epsilon^2h) - f_i(\vec{r}_1,t_1,t_2) = \delta_i
\end{equation*}

* Taylor expansion:

\begin{equation*}
\epsilon h \left( \frac{\partial}{\partial t_1} + \vec{c}_i\cdot\frac{\partial}{\partial\vec{r}_1} \right) f_i + \epsilon^2h \left[ \frac{\partial}{\partial t_2} + \frac{h}{2} \left( \frac{\partial}{\partial t_1} + \vec{c}_i\cdot\frac{\partial}{\partial\vec{r}_1}\right) \right] f_i = \delta_i
\end{equation*}

###  Chapman-Enskog expansion for LB

* expand $f_i$ and $\Delta_i$ in powers of $\epsilon$
\begin{eqnarray*}
f_i &=& f_i^{(0)} + \epsilon f_i^{(1)} + \epsilon^2 f_i^{(2)} + \dots \\
\Delta_i &=& \Delta_i^{(0)} + \epsilon \Delta_i^{(1)} + \dots
%&=& \Delta_i^{(0)} + \epsilon \sum_j \textcolor{blue}{\left. \frac{\partial \Delta_i}{\partial f_j}\right|_{f_j=f_j^{(0)}}} f_j^{(1)} + \dots \\
%&=:& \epsilon \sum_j \textcolor{blue}{\Lambda_{ij}} f_j^{(1)}
\end{eqnarray*}

* hierarchy of equations at different powers of $\epsilon$
\begin{align*}
\mathcal{O}(\epsilon^0): \quad & \Delta_i^{(0)} = 0 \\
\mathcal{O}(\epsilon^1): \quad & \left( \frac{\partial}{\partial t_1} + \vec{c}_i\cdot\frac{\partial}{\partial\vec{r}_1} \right) f_i^{(0)} = \frac{1}{h} \Delta_i^{(1)} \\
\mathcal{O}(\epsilon^2): \quad &  \left[ \frac{\partial}{\partial t_2} + \frac{h}{2} \left(  \frac{\partial}{\partial t_1} + \vec{c}_i\cdot\frac{\partial}{\partial\vec{r}_1} \right)^2 \right]  f_i^{(0)}  +  \left( \frac{\partial}{\partial t_1} + \vec{c}_i\cdot\frac{\partial}{\partial\vec{r}_1} \right) f_i^{(1)} = \frac{1}{h} \Delta_i^{(2)}
\end{align*}

### Zeroth order $\epsilon^0$

* no expansion for conserved quantities!
\begin{eqnarray*}
f_i^{(0)} &=& f_i^\text{eq} \\
\rho^{(0)} &=& \rho = \sum_i f_i^\text{eq} \\
\vec{j}^{(0)} &=& \vec{j} = \sum_i f_i^\text{eq} \vec{c}_i
\end{eqnarray*}

* linear part of collision operator
\begin{eqnarray*}
\Delta_i &=& \epsilon \Delta_i^{(1)} = \epsilon \sum_j {\left. \frac{\partial \Delta_i}{\partial f_j} \right|_{f^{(0)}}} f_j^{(1)} = \sum_j {\Lambda_{ij}} f_j^{(1)}
\end{eqnarray*}

### Equations for the mass density

* 
\begin{align*}
&\frac{\partial}{\partial t_1} \rho + \frac{\partial}{\partial\vec{r}_1} \cdot \vec{j} = 0 \\
&\frac{\partial}{\partial t_2} \rho = 0
\end{align*}

* continuity equation holds! \centering

### Equations for the momentum density

* 
\begin{align*}
&\frac{\partial}{\partial t_1} \vec{j} + \frac{\partial}{\partial\vec{r}_1} \cdot {\mathsf{\Pi}^{(0)}} = 0 \\
&\frac{\partial}{\partial t_2} \vec{j} + \frac{1}{2} \frac{\partial}{\partial\vec{r}_1} \cdot \left( {\mathsf{\Pi}^{*(1)} + \mathsf{\Pi}^{(1)}} \right) = 0 
\end{align*}

* Euler stress
\begin{equation*}
{\Pi}^{(0)} = \rho c_s^2 \mathsf{I} + \rho \vec{u}\vec{u} = \mathsf{\Pi}^\text{eq}
\end{equation*}

* Newtonian viscous stress
\begin{equation*}
\frac{\epsilon}{2} \left( \mathsf{\Pi}^{*(1)} + \mathsf{\Pi}^{(1)} \right) = - \mathsf{\Pi}^\text{visc}
\end{equation*}

* incompressible Navier-Stokes equation holds!

### Diggin' deeper: Third moment

* the third moment ${\Phi}^{(0)} = \sum_i f_i^{(0)} \vec{c}_i\vec{c}_i\vec{c}_i$ enters through its equilibrium part!
\begin{equation*}
\frac{\partial}{\partial t_1} \mathsf{\Pi}^{(0)} + \frac{\partial}{\partial\vec{r}_1} \cdot {{\Phi}^{(0)}} = \frac{1}{h} \sum_i \Delta_i^{(1)} \vec{c}_i \vec{c}_i = \frac{1}{h} \left( \mathsf{\Pi}^{*(1)} - \mathsf{\Pi}^{(1)} \right)
\end{equation*}

* explicit form
\begin{equation*}
{\Phi}^{(0)}_{\alpha\beta\gamma} = \rho c_s^2 \left( u_\alpha \delta_{\beta\gamma} + u_\beta\delta_{\alpha\gamma} + u_\gamma\delta_{\alpha\beta} \right)
\end{equation*}

* from continuity and Euler equation
\begin{equation*}
\frac{\partial}{\partial t_1} \mathsf{\Pi}^{(0)} = \frac{\partial}{\partial t_1} \left( \rho c_s^2 \mathsf{I} + \rho \vec{u}\vec{u} \right) = \dots
\end{equation*}

* neglecting terms of $\mathcal{O}(u^3)$
\begin{equation*}
\mathsf{\Pi}^{*(1)} - \mathsf{\Pi}^{(1)} = \rho c_s^2 h \left( \nabla\vec{u} + (\nabla\vec{u})^t \right)
\end{equation*}

### Suitable LB models

* equilibrium values of the moments up to ${\Phi}^\text{eq}$
\begin{eqnarray*}
\rho^\text{eq}&=&\rho \\
\vec{j}^\text{eq}&=&\vec{j} \\
{\Pi}^\text{eq}&=&\rho c_s^2 \mathsf{I} + \rho \vec{u} \vec{u} \\
{\Phi}^\text{eq}_{\alpha\beta\gamma} &=& \rho c_s^2 \left( u_\alpha \delta_{\beta\gamma} + u_\beta\delta_{\alpha\gamma} + u_\gamma\delta_{\alpha\beta} \right)
\end{eqnarray*}

* collision operator
\begin{align*}
\sum_i \Delta_i &= 0 & 
\sum_i \Delta_i \vec{c}_i &= 0 \\
\overline{{\Pi}}^{*\text{neq}} &= \gamma_s \overline{{\Pi}}^{\text{neq}} &
\text{tr}({\Pi}^{*\text{neq}}) &= \gamma_b \text{tr}({\Pi}^{\text{neq}})
\end{align*}

### Hands-On Activity

1. Play around with the LB code available at https://gist.github.com/uschille/8f65dd40572b2d943409.

### References

1. Schiller, U. D., Krüger, T. & Henrich, O. Mesoscopic modelling and simulation of soft matter. Soft Matter 14, 9–26 (2017).
4. Schiller, U. D. A unified operator splitting approach for multi-scale fluid–particle coupling in the lattice Boltzmann method. Comput. Phys. Commun. 185, 2586–2597 (2014).
2. Dünweg, B., Schiller, U. D. & Ladd, A. J. C. Progress in the understanding of the fluctuating lattice Boltzmann equation. Comput. Phys. Commun. 180, 605–608 (2009).
5. Dünweg, B., Schiller, U. D. & Ladd, A. J. C. Statistical mechanics of the fluctuating lattice Boltzmann equation. Phys. Rev. E 76, 036704 (2007).
3. Schiller, U. D. Thermal fluctuations and boundary conditions in the lattice Boltzmann method. (Johannes Gutenberg University Mainz, 2008).